In [11]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [12]:
pd.__version__

'1.4.2'

In [13]:
# Q1. Download the data for January and February 2023

df1=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df2=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [14]:
# Read the data for January. How many columns are there? Answer: 19
len(df1.columns)

19

In [15]:
# Create duration column
df1['duration'] = df1.tpep_dropoff_datetime - df1.tpep_pickup_datetime
df1.duration = df1.duration.apply(lambda td: td.total_seconds() / 60)

In [16]:
# Calculate standard deviation
std_duration = df1['duration'].std()

# Q2: What's the standard deviation of the trips duration in January?
# Answer: 42.59
print("Standard Deviation of Duration:", std_duration)

Standard Deviation of Duration: 42.594351241920904


In [17]:
df = df1[(df1.duration >= 1) & (df1.duration <= 60)]

In [18]:
# Q3: What fraction of the records left after you dropped the outliers?
# Answer: 98%
fraction = len(df) / len(df1)
print("fraction of the records left after you dropped the outliers:", fraction)

fraction of the records left after you dropped the outliers: 0.9812202822125979


In [19]:
categorical = ['PULocationID', 'DOLocationID']
df = df.copy()
df[categorical] = df[categorical].astype(str)

In [20]:
df[categorical][:5].to_dict(orient='records')

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [21]:
train_dicts = df[categorical].to_dict(orient='records')

In [22]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [23]:
# Q4: apply one-hot encoding to the pickup and dropoff location IDs.
X_train.shape

# What's the dimensionality of this matrix (number of columns)?
# answer: 515

(3009173, 515)

In [24]:
y_train = df['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [25]:
# Q5: What's the RMSE on train?
# Answer: 7.64
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(rmse)

7.6492610279057605


In [26]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [27]:
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [28]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [30]:
y_val = df_val['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

In [31]:
# Q6: What's the RMSE on validation?
# Answer: 7.81
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(rmse)

7.81183265470218
